<a href="https://colab.research.google.com/github/AyanBhardwaj06/Ayan_projects/blob/main/Multi_constaraints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiple Constraints

# Periphery & Non-adj

In [ ]:
import random

def main():
    def is_valid(grid, row, col, x, y, color):
        directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # Right, Down, Left, Up
        return all(0 <= x + dx < row and 0 <= y + dy < col and
                   (grid[x + dx][y + dy] != color or grid[x + dx][y + dy] == ' ')
                   for dx, dy in directions)

    def fill_periphery(grid, row, col, periphery_order, color_counts):
        positions = []

        for i in range(col):
            positions.append((0, i))  # Top row
            positions.append((row - 1, i))  # Bottom row
        for i in range(1, row - 1):
            positions.append((i, 0))  # Left column
            positions.append((i, col - 1))  # Right column

        random.shuffle(positions)

        for color in periphery_order:
            for x, y in positions:
                if grid[x][y] == ' ' and color_counts[color] > 0:
                    grid[x][y] = color
                    color_counts[color] -= 1

        return grid, color_counts

    def fill_non_adjacent(grid, row, col, color_counts):
        layer = 0
        placed_positions = []

        while sum(color_counts.values()) > 0:
            positions = []
            for i in range(layer, col - layer):
                positions.append((layer, i))
                positions.append((row - 1 - layer, i))
            for i in range(layer + 1, row - 1 - layer):
                positions.append((i, layer))
                positions.append((i, col - 1 - layer))

            random.shuffle(positions)

            for x, y in positions:
                if grid[x][y] == ' ':
                    placed = False
                    color_order = random.sample(['R', 'G', 'B'], 3)
                    for color in color_order:
                        if color_counts[color] > 0 and is_valid(grid, row, col, x, y, color):
                            grid[x][y] = color
                            color_counts[color] -= 1
                            placed_positions.append((x, y, color))
                            placed = True
                            break

                    if not placed and placed_positions:
                        last_x, last_y, last_color = placed_positions.pop()
                        grid[last_x][last_y] = ' '
                        color_counts[last_color] += 1

                        retry_colors = [c for c in ['R', 'G', 'B'] if c != last_color and color_counts[c] > 0]
                        random.shuffle(retry_colors)

                        retry_success = False
                        for new_color in retry_colors:
                            if is_valid(grid, row, col, last_x, last_y, new_color):
                                grid[last_x][last_y] = new_color
                                color_counts[new_color] -= 1
                                placed_positions.append((last_x, last_y, new_color))
                                retry_success = True
                                break

                        if not retry_success:
                            grid[last_x][last_y] = last_color
                            color_counts[last_color] -= 1
                            placed_positions.append((last_x, last_y, last_color))

                if grid[x][y] == ' ':
                    retry_colors = [c for c in ['R', 'G', 'B'] if color_counts[c] > 0]
                    random.shuffle(retry_colors)

                    retry_success = False
                    for new_color in retry_colors:
                        if is_valid(grid, row, col, x, y, new_color):
                            grid[x][y] = new_color
                            color_counts[new_color] -= 1
                            placed_positions.append((x, y, new_color))
                            retry_success = True
                            break

                    if not retry_success:
                        max_color = max(color_counts, key=color_counts.get)
                        if color_counts[max_color] > 0:
                            grid[x][y] = max_color
                            color_counts[max_color] -= 1
                            placed_positions.append((x, y, max_color))

            layer += 1
        return grid

    row = int(input("Enter number of rows: "))
    col = int(input("Enter number of columns: "))
    red = int(input("Enter number of red cells: "))
    green = int(input("Enter number of green cells: "))
    blue = int(input("Enter number of blue cells: "))

    if row * col != red + green + blue:
        print("Error: The total number of colored cells does not match the grid size!")
        return

    periphery_order = input("Enter the order of periphery colors (space-separated, e.g., G R B): ").split()
    non_adjacent = input("Do you want the grid to be non-adjacent? (y/n): ").strip().lower()

    grid = [[' ' for _ in range(col)] for _ in range(row)]
    color_counts = {'R': red, 'G': green, 'B': blue}

    grid, color_counts = fill_periphery(grid, row, col, periphery_order, color_counts)

    if non_adjacent == 'y':
        grid = fill_non_adjacent(grid, row, col, color_counts)
    else:
        empty_positions = [(i, j) for i in range(row) for j in range(col) if grid[i][j] == ' ']
        random.shuffle(empty_positions)
        for x, y in empty_positions:
            for color in list(color_counts.keys()):
                if color_counts[color] > 0:
                    grid[x][y] = color
                    color_counts[color] -= 1
                    break

    for r in grid:
        print(' '.join(r))

if __name__ == "__main__":
    main()


# Periphery & Big Colour

In [ ]:
import random

def main():
    def fill_periphery(grid, row, col, periphery_order, color_counts):
        positions = [(0, i) for i in range(col)] + [(row - 1, i) for i in range(col)] + [(i, 0) for i in range(1, row - 1)] + [(i, col - 1) for i in range(1, row - 1)]
        random.shuffle(positions)
        for color in periphery_order:
            for x, y in positions:
                if grid[x][y] == ' ' and color_counts.get(color, 0) > 0:
                    grid[x][y] = color
                    color_counts[color] -= 1
        return grid, color_counts

    def place_blocks(grid, block_color, block_size, block_count, n, m, color_counts):
        required_tiles = block_count * (block_size ** 2)
        if required_tiles > color_counts[block_color]:
            print(f"Error: Not enough {block_color} tiles to form {block_count} blocks of size {block_size}×{block_size}.")
            return grid, color_counts, False
        placed_blocks = 0
        row = 1
        while row + block_size - 1 < n - 1 and placed_blocks < block_count:
            col = 1
            while col + block_size - 1 < m - 1 and placed_blocks < block_count:
                if all(grid[r][c] == ' ' for r in range(row, row + block_size) for c in range(col, col + block_size)):
                    for r in range(row, row + block_size):
                        for c in range(col, col + block_size):
                            grid[r][c] = block_color
                    placed_blocks += 1
                    color_counts[block_color] -= (block_size ** 2)
                col += block_size
            row += block_size
        if placed_blocks < block_count:
            print(f"Warning: Could only place {placed_blocks} out of {block_count} blocks.")
            return grid, color_counts, False
        return grid, color_counts, True

    def fill_remaining(grid, color_counts, n, m):
        empty_positions = [(row, col) for row in range(n) for col in range(m) if grid[row][col] == ' ']
        random.shuffle(empty_positions)
        for (row, col) in empty_positions:
            available_colors = [color for color in color_counts if color_counts[color] > 0]
            if available_colors:
                chosen_color = random.choice(available_colors)
                grid[row][col] = chosen_color
                color_counts[chosen_color] -= 1
        return grid

    row = int(input("Enter number of rows: "))
    col = int(input("Enter number of columns: "))
    red = int(input("Enter number of red cells: "))
    green = int(input("Enter number of green cells: "))
    blue = int(input("Enter number of blue cells: "))

    if row * col != red + green + blue:
        print("Error: The total number of colored cells does not match the grid size!")
        return

    periphery_order = input("Enter the order of periphery colors (space-separated, e.g., G R B): ").split()
    grid = [[' ' for _ in range(col)] for _ in range(row)]
    color_counts = {'R': red, 'G': green, 'B': blue}
    grid, color_counts = fill_periphery(grid, row, col, periphery_order, color_counts)

    block_color = input("Enter block color (R/G/B): ").strip().upper()
    block_size = int(input("Enter block size: "))
    block_count = int(input("Enter number of blocks: "))
    grid, color_counts, success = place_blocks(grid, block_color, block_size, block_count, row, col, color_counts)
    grid = fill_remaining(grid, color_counts, row, col)

    print("\nFinal Grid:")
    for r in grid:
        print(' '.join(r))
    if not success:
        print("Some blocks could not be placed.")

if __name__ == "__main__":
    main()


Enter number of rows: 9
Enter number of columns: 9
Enter number of red cells: 26
Enter number of green cells: 30
Enter number of blue cells: 25
Enter the order of periphery colors (space-separated, e.g., G R B): R
Enter block color (R/G/B): B
Enter block size: 2
Enter number of blocks: 3

Final Grid:
G R R R R R R R R
R B B B B B B G R
R B B B B B B G R
R B G G G G G G R
R B G G G G G B G
G B G G G G B G R
R B G B G G B B R
R G B G G B G B R
R G R R R R B R G


# Periphery & Pattern

In [ ]:
import random

def generate_grid(row, col, red, green, blue, periphery_order, pattern_length, pattern):
    def max_patterns_count(color_counts, pattern):
        pattern_color_counts = {color: pattern.count(color) for color in set(pattern)}
        return min(
            (color_counts.get(color, 0) // count if count > 0 else float('inf'))
            for color, count in pattern_color_counts.items()
        )

    def fill_remaining(grid, color_counts):
        empty_positions = [(r, c) for r in range(row) for c in range(col) if grid[r][c] == ' ']
        random.shuffle(empty_positions)
        for r, c in empty_positions:
            available_colors = [color for color in color_counts if color_counts[color] > 0]
            if available_colors:
                chosen_color = random.choice(available_colors)
                grid[r][c] = chosen_color
                color_counts[chosen_color] -= 1

    def fill_with_patterns(grid, pattern, max_patterns_count):
        applied_patterns = 0
        for r in range(1, row - 1):
            for start_col in range(0, col, pattern_length):
                end_col = start_col + pattern_length
                if end_col > col or applied_patterns >= max_patterns_count:
                    continue
                if all(grid[r][c] == ' ' for c in range(start_col, end_col)):
                    for i, color in enumerate(pattern):
                        grid[r][start_col + i] = color
                        color_counts[color] -= 1
                    applied_patterns += 1

    def fill_periphery(grid):
        positions = [(0, i) for i in range(col)] + [(row - 1, i) for i in range(col)] + [(i, 0) for i in range(1, row - 1)] + [(i, col - 1) for i in range(1, row - 1)]
        random.shuffle(positions)
        for color in periphery_order:
            for x, y in positions:
                if grid[x][y] == ' ' and color_counts.get(color, 0) > 0:
                    grid[x][y] = color
                    color_counts[color] -= 1

    grid = [[' ' for _ in range(col)] for _ in range(row)]
    color_counts = {'R': red, 'G': green, 'B': blue}

    if max_patterns_count(color_counts, pattern) == 0:
        return None  # Only return used here ✅

    fill_periphery(grid)
    fill_with_patterns(grid, pattern, max_patterns_count(color_counts, pattern))
    fill_remaining(grid, color_counts)

    return grid  # 🔹 The only return statement


# Input handling
row = int(input("Enter number of rows: "))
col = int(input("Enter number of columns: "))
red = int(input("Enter number of red cells: "))
green = int(input("Enter number of green cells: "))
blue = int(input("Enter number of blue cells: "))

if row * col != red + green + blue:
    print("Error: The total number of colored cells does not match the grid size!")
else:
    periphery_order = input("Enter the order of periphery colors (space-separated, e.g., G R B): ").split()
    pattern_length = int(input("Enter the number of tiles for the pattern: "))
    pattern = input(f"Enter the pattern of {pattern_length} tiles (e.g., RRGB): ").strip().upper()

    if pattern_length > col:
        print("\n❌ This configuration cannot be possible! The pattern length exceeds the number of columns.")
    elif len(pattern) != pattern_length:
        print("\n❌ Invalid Pattern: The specified pattern length does not match the given pattern!")
    else:
        grid = generate_grid(row, col, red, green, blue, periphery_order, pattern_length, pattern)
        if grid:
            for r in grid:
                print(' '.join(r))
        else:
            print("\n❌ This configuration cannot be possible!")


Enter number of rows: 8
Enter number of columns: 9
Enter number of red cells: 23
Enter number of green cells: 25
Enter number of blue cells: 24
Enter the order of periphery colors (space-separated, e.g., G R B): B
Enter the number of tiles for the pattern: 3
Enter the pattern of 3 tiles (e.g., RRGB): RRG
B B B B B B B B B
B R R R R G R R G
B G G R R G G G B
B G G R R G R R G
B G G R R G G G B
B G G R R G R R G
B R G R R G R G B
G G R B B B B B B


# Diagonal and pheriphery constraint (pattern)

In [ ]:
import random

def fill_grid_combined(dimension, red, green, blue, periphery_colors=None, diagonal_colors=None):
    grid = [[None] * dimension for _ in range(dimension)]
    color_counts = {'R': red, 'G': green, 'B': blue}

    periphery_positions = []
    for j in range(dimension):
        periphery_positions.append((0, j))
    for i in range(1, dimension):
        periphery_positions.append((i, dimension - 1))
    for j in range(dimension - 2, -1, -1):
        periphery_positions.append((dimension - 1, j))
    for i in range(dimension - 2, 0, -1):
        periphery_positions.append((i, 0))

    main_diagonal = [(i, i) for i in range(dimension)]
    anti_diagonal = [(i, dimension - 1 - i) for i in range(dimension)]
    all_diagonal_positions = main_diagonal + anti_diagonal

    diagonal_index = 0
    for pos in all_diagonal_positions:
        i, j = pos
        if grid[i][j] is not None:
            continue
        if diagonal_colors and diagonal_index < len(diagonal_colors) and color_counts[diagonal_colors[diagonal_index]] > 0:
            grid[i][j] = diagonal_colors[diagonal_index]
            color_counts[diagonal_colors[diagonal_index]] -= 1
        else:
            for color in 'RGB':
                if color_counts[color] > 0:
                    grid[i][j] = color
                    color_counts[color] -= 1
                    break
        diagonal_index = (diagonal_index + 1) % len(diagonal_colors) if diagonal_colors else 0

    periphery_index = 0
    for pos in periphery_positions:
        i, j = pos
        if grid[i][j] is not None:
            continue
        if periphery_colors and periphery_index < len(periphery_colors) and color_counts[periphery_colors[periphery_index]] > 0:
            grid[i][j] = periphery_colors[periphery_index]
            color_counts[periphery_colors[periphery_index]] -= 1
        else:
            for color in 'RGB':
                if color_counts[color] > 0:
                    grid[i][j] = color
                    color_counts[color] -= 1
                    break
        periphery_index = (periphery_index + 1) % len(periphery_colors) if periphery_colors else 0

    for i in range(dimension):
        for j in range(dimension):
            if grid[i][j] is None:
                for color in 'RGB':
                    if color_counts[color] > 0:
                        grid[i][j] = color
                        color_counts[color] -= 1
                        break

    return grid


# Input is taken outside and passed to the function
dimension = int(input("Enter the dimension of the square grid: "))
total_cells = dimension * dimension

while True:
    red = int(input("Enter the number of red tiles: "))
    green = int(input("Enter the number of green tiles: "))
    blue = int(input("Enter the number of blue tiles: "))
    if red + green + blue == total_cells:
        break
    print(f"Error: The sum of tiles must equal {total_cells} (dimension²). Please try again.")

use_periphery = input("Apply periphery constraint? (y/n): ").lower() == 'y'
use_diagonal = input("Apply diagonal constraint? (y/n): ").lower() == 'y'

periphery_colors = None
diagonal_colors = None

if use_periphery:
    periphery_colors = input("Enter the color pattern for the periphery in order (R/G/B, e.g., R G B): ").strip().upper().split()
if use_diagonal:
    diagonal_colors = input("Enter the color pattern for the diagonal in order (R/G/B, e.g., R G B): ").strip().upper().split()

grid = fill_grid_combined(dimension, red, green, blue, periphery_colors, diagonal_colors)

# Print the grid
for row in grid:
    print(" ".join(row))


Enter the dimension of the square grid: 6
Enter the number of red tiles: 12
Enter the number of green tiles: 15
Enter the number of blue tiles: 9
Apply periphery constraint? (y/n): y
Apply diagonal constraint? (y/n): y
Enter the color pattern for the periphery in order (R/G/B, e.g., R G B): G
Enter the color pattern for the diagonal in order (R/G/B, e.g., R G B): R B
R G G G G R
R B R R B G
G R R R R G
G R B B B G
G R B B R G
B G G G G B


# Periphery and Diagonal constraint (priority)

In [ ]:
import random

def fill_grid_combined(dimension, red, green, blue, periphery_colors=None, diagonal_colors=None, constraint_priority="diagonal"):
    def initialize_grid():
        return [[None] * dimension for _ in range(dimension)]

    def get_positions():
        periphery = [(0, j) for j in range(dimension)] + \
                    [(i, dimension - 1) for i in range(1, dimension)] + \
                    [(dimension - 1, j) for j in range(dimension - 2, -1, -1)] + \
                    [(i, 0) for i in range(dimension - 2, 0, -1)]
        diagonal = list(set([(i, i) for i in range(dimension)] + [(i, dimension - 1 - i) for i in range(dimension)]))
        return periphery, diagonal

    def assign_priority():
        return [(diagonal_positions, diagonal_colors), (periphery_positions, periphery_colors)] if constraint_priority == "diagonal" else [(periphery_positions, periphery_colors), (diagonal_positions, diagonal_colors)]

    def fill_positions(priority_order):
        for positions, color_priority in priority_order:
            if not color_priority:
                continue
            for i, j in positions:
                if grid[i][j] is not None:
                    continue
                assigned = False
                for color in color_priority:
                    if color_counts[color] > 0:
                        grid[i][j] = color
                        color_counts[color] -= 1
                        assigned = True
                        break
                if not assigned:
                    for color in 'RGB':
                        if color_counts[color] > 0:
                            grid[i][j] = color
                            color_counts[color] -= 1
                            break

    def fill_remaining():
        for i in range(dimension):
            for j in range(dimension):
                if grid[i][j] is None:
                    for color in 'RGB':
                        if color_counts[color] > 0:
                            grid[i][j] = color
                            color_counts[color] -= 1
                            break

    grid = initialize_grid()
    color_counts = {'R': red, 'G': green, 'B': blue}
    periphery_positions, diagonal_positions = get_positions()
    priority_order = assign_priority()
    fill_positions(priority_order)
    fill_remaining()
    return grid

# Taking input outside the function
dimension = int(input("Enter the dimension of the square grid: "))
total_cells = dimension * dimension
while True:
    red = int(input("Enter the number of red tiles: "))
    green = int(input("Enter the number of green tiles: "))
    blue = int(input("Enter the number of blue tiles: "))
    if red + green + blue == total_cells:
        break
    print(f"Error: The sum of tiles must equal {total_cells} (dimension²). Please try again.")

use_periphery = input("Apply periphery constraint? (y/n): ").lower() == 'y'
use_diagonal = input("Apply diagonal constraint? (y/n): ").lower() == 'y'

periphery_colors = None
diagonal_colors = None
constraint_priority = "diagonal"
if use_periphery and use_diagonal:
    priority_choice = input("Which constraint has priority? (d)iagonal or (p)eriphery: ").lower()
    constraint_priority = "diagonal" if priority_choice.startswith('d') else "periphery"

if use_periphery:
    periphery_colors = list(dict.fromkeys(input("Enter periphery colors (R G B): ").strip().upper().split()))
if use_diagonal:
    diagonal_colors = list(dict.fromkeys(input("Enter diagonal colors (R G B): ").strip().upper().split()))

# Generating and printing the grid
grid = fill_grid_combined(dimension, red, green, blue, periphery_colors, diagonal_colors, constraint_priority)
for row in grid:
    print(" ".join(row))


# Diagonal Constraint & Adjacency Constraint

In [ ]:
import random

def generate_colored_grid(dimension, red, green, blue, diagonal_colors, adjacent_tiles):
    def fill_grid():
        total_cells = dimension * dimension
        total_colors = red + green + blue

        if total_colors != total_cells:
            print("Error: Incorrect number of colors.")
            return None

        color_counts.update({'R': red, 'G': green, 'B': blue})
        main_diagonal = [(i, i) for i in range(dimension)]
        anti_diagonal = [(i, dimension - 1 - i) for i in range(dimension)]
        all_diagonal_positions = list(dict.fromkeys(main_diagonal + anti_diagonal))
        random.shuffle(all_diagonal_positions)

        for i, j in all_diagonal_positions:
            filled = False
            for priority_color in diagonal_colors:
                if color_counts[priority_color] > 0:
                    grid[i][j] = priority_color
                    color_counts[priority_color] -= 1
                    filled = True
                    break

            if not filled:
                available_colors = [c for c in 'RGB' if color_counts[c] > 0]
                random.shuffle(available_colors)
                for color in available_colors:
                    if color_counts[color] > 0:
                        grid[i][j] = color
                        color_counts[color] -= 1
                        break

    def violates_adjacency(row, col, tile):
        adjacent_positions = [(row - 1, col), (row + 1, col), (row, col - 1), (row, col + 1)]
        other_colors = {'R', 'G', 'B'} - set(adjacent_tiles)
        same_adjacent_count = 0

        for r, c in adjacent_positions:
            if 0 <= r < dimension and 0 <= c < dimension:
                if grid[r][c] == tile:
                    return True
                if grid[r][c] in other_colors:
                    same_adjacent_count += 1

        return same_adjacent_count >= 2

    def fill_with_adjacency():
        tile1, tile2 = sorted(adjacent_tiles, key=lambda x: color_counts[x])
        tile1_positions = []
        empty_positions = [(i, j) for i in range(dimension) for j in range(dimension) if grid[i][j] is None]
        random.shuffle(empty_positions)

        for i, j in empty_positions[:]:
            if color_counts[tile1] > 0 and not violates_adjacency(i, j, tile1):
                grid[i][j] = tile1
                color_counts[tile1] -= 1
                tile1_positions.append((i, j))
                empty_positions.remove((i, j))
                adjacent_positions = [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]
                random.shuffle(adjacent_positions)

                for ai, aj in adjacent_positions:
                    if (0 <= ai < dimension and 0 <= aj < dimension and grid[ai][aj] is None and color_counts[tile2] > 0):
                        grid[ai][aj] = tile2
                        color_counts[tile2] -= 1
                        empty_positions.remove((ai, aj))

        for i, j in tile1_positions:
            diagonal_positions = [(i-1, j-1), (i-1, j+1), (i+1, j-1), (i+1, j+1)]
            for di, dj in diagonal_positions:
                if (0 <= di < dimension and 0 <= dj < dimension and grid[di][dj] is None and color_counts[tile1] > 0):
                    grid[di][dj] = tile1
                    color_counts[tile1] -= 1
                    empty_positions.remove((di, dj))
                    if color_counts[tile1] == 0:
                        break
            if color_counts[tile1] == 0:
                break

        remaining_tiles = [tile for tile, count in color_counts.items() for _ in range(count)]
        random.shuffle(remaining_tiles)

        for i, j in empty_positions:
            if remaining_tiles:
                grid[i][j] = remaining_tiles.pop()

    grid = [[None] * dimension for _ in range(dimension)]
    color_counts = {}

    fill_grid()
    if None in [tile for row in grid for tile in row]:
        fill_with_adjacency()

    return grid

# Take input outside the function
dimension = int(input("Enter the dimension: "))
red = int(input("Enter the number of red tiles: "))
green = int(input("Enter the number of green tiles: "))
blue = int(input("Enter the number of blue tiles: "))
diagonal_colors = input("Enter the colors for the diagonal in order (R G B): ").strip().upper().split()
adjacent_tiles = input("Enter two tiles for adjacency constraint (e.g., R G): ").strip().upper().split()

# Run the function
grid = generate_colored_grid(dimension, red, green, blue, diagonal_colors, adjacent_tiles)
print("Generated Grid:")
for row in grid:
    print(" ".join(row))


Enter the dimension: 9
Enter the number of red tiles: 25
Enter the number of green tiles: 26
Enter the number of blue tiles: 30
Enter the colors for the diagonal in order (R G B): G
Enter two tiles for adjacency constraint (e.g., R G): R B
Generated Grid:
G B R B R B R B G
B G B R B R B G G
R B G B R B G B R
B R B G G G B R B
R B R G G G R B B
B R B G G G G B R
R B G B R G G R B
G G B R B R G G R
G B R B R B R R G
